#  Sentiment Analysis Using SpaCy

In [1]:
import spacy
from spacy import displacy

In [2]:
nlp = spacy.load('en_core_web_sm')

In [12]:
text = "Apple, This is first sentence. and Google this is another one. here 3rd one is"
doc = nlp(text)
doc

Apple, This is first sentence. and Google this is another one. here 3rd one is

In [13]:
for token in doc:
    print(token)

Apple
,
This
is
first
sentence
.
and
Google
this
is
another
one
.
here
3rd
one
is


In [14]:
doc = nlp(text)

In [15]:
for sent in doc.sents:
    print(sent)

Apple, This is first sentence.
and Google this is another one.
here 3rd one is


In [19]:
from spacy.lang.en.stop_words import STOP_WORDS
stopwords = list(STOP_WORDS)
print(stopwords)

['now', 'eleven', 'nowhere', 'always', 'amount', 'within', 'in', 'upon', 'anyhow', 'down', 'get', 'only', 'unless', 'anything', 'top', 'are', 'her', 'their', 'whereafter', 'thereafter', 'by', 'been', '‘m', 'never', 'every', 'itself', 'anyone', 'but', 'my', 'off', "'ve", 'whenever', 'doing', 'above', 'how', 'too', 'everywhere', 'indeed', 'any', 'onto', 'through', 'nor', 'just', 'should', 'eight', 'otherwise', 'thereby', 'also', 'i', "'d", 'others', 'moreover', 'which', 'yourselves', 'some', 'often', 'whose', 'elsewhere', 'per', 'used', 'thereupon', 'when', 'show', 'several', 'empty', 'beyond', 'whereas', 'hundred', 'at', 'almost', 'behind', 'it', 'being', 'or', 'would', 'done', 'sometime', 'six', 'such', 'ca', 'already', 'you', 'most', 'seems', 'everyone', 'everything', 'alone', 'whereby', 'mine', 'five', 'give', 'keep', 'himself', 'third', 'along', '‘re', 'among', 'while', 'about', 'via', 'enough', 'other', 'across', 'hereafter', 'name', 'another', 'using', 'thru', 'under', "'re", 'thi

In [20]:
len(stopwords)

326

In [21]:
for token in doc:
    if token.is_stop == False:
        print(token)

Apple
,
sentence
.
Google
.
3rd


### Lemmatization 

In [22]:
doc = nlp('run runs running runner')

In [23]:
for lem in doc:
    print(lem.text, lem.lemma_)

run run
runs run
running run
runner runner


### POS 

In [24]:
doc = nlp('All is well at your end!')

In [25]:
for token in doc:
    print(token.text, token.pos_)

All PRON
is AUX
well ADV
at ADP
your PRON
end NOUN
! PUNCT


In [26]:
displacy.render(doc, style = 'dep')

### Entity Detection 

In [28]:
doc = nlp("New York City on Tuesday declared a public health emergency and ordered mandatory measles vaccinations amid an outbreak, becoming the latest national flash point over refusals to inoculate against dangerous diseases. At least 285 people have contracted measles in the city since September, mostly in Brooklyn’s Williamsburg neighborhood. The order covers four Zip codes there, Mayor Bill de Blasio (D) said Tuesday. The mandate orders all unvaccinated people in the area, including a concentration of Orthodox Jews, to receive inoculations, including for children as young as 6 months old. Anyone who resists could be fined up to $1,000.")
displacy.render(doc, style = 'ent')

### Text Classification 

In [29]:
import pandas as pd
df= pd.read_csv('IMDB Dataset.csv')
df.head()

,review,sentiment
0,One of the other reviewers has mentioned that ...,positive
1,A wonderful little production. <br /><br />The...,positive
2,I thought this was a wonderful way to spend ti...,positive
3,Basically there's a family where a little boy ...,negative
4,"Petter Mattei's ""Love in the Time of Money"" is...",positive


In [30]:
df.shape

(50000, 2)

In [32]:
df['sentiment'].value_counts()

sentiment
positive    25000
negative    25000
Name: count, dtype: int64

In [34]:
df.isnull().sum()

review       0
sentiment    0
dtype: int64

### Tokenization 

In [35]:
import string
punct = string.punctuation
punct

'!"#$%&\'()*+,-./:;<=>?@[\\]^_`{|}~'

In [36]:
def text_data_cleaning(sentence):
    doc = nlp(sentence)
    
    tokens = []
    for token in doc:
        if token.lemma_ != "-PRON-":
            temp = token.lemma_.lower().strip()
        else:
            temp = token.lower_
        tokens.append(temp)
    
    cleaned_tokens = []
    for token in tokens:
        if token not in stopwords and token not in punct:
            cleaned_tokens.append(token)
    return cleaned_tokens

In [37]:
text_data_cleaning("    Hello how are you. Like this video")

['hello', 'like', 'video']

In [39]:
X = df['review']
y = df['sentiment']

In [42]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2, random_state = 42)
X_train.shape, X_test.shape

((40000,), (10000,))

In [43]:
from sklearn.feature_extraction.text import TfidfVectorizer
tfidf = TfidfVectorizer(tokenizer = text_data_cleaning)

from sklearn.svm import LinearSVC
classifier = LinearSVC()

from sklearn.pipeline import Pipeline
clf = Pipeline([('tfidf', tfidf), ('clf', classifier)])

In [44]:
clf.fit(X_train, y_train)

c:\Users\Razil\.conda\envs\DL\lib\site-packages\sklearn\feature_extraction\text.py:525: UserWarning: The parameter 'token_pattern' will not be used since 'tokenizer' is not None'
  warnings.warn(
c:\Users\Razil\.conda\envs\DL\lib\site-packages\sklearn\svm\_classes.py:32: FutureWarning: The default value of `dual` will change from `True` to `'auto'` in 1.5. Set the value of `dual` explicitly to suppress the warning.
  warnings.warn(


Pipeline(steps=[('tfidf',
                 TfidfVectorizer(tokenizer=<function text_data_cleaning at 0x000001E3D9270F70>)),
                ('clf', LinearSVC())])

In [45]:
y_pred = clf.predict(X_test)

In [46]:
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix
print(confusion_matrix(y_test, y_pred))

[[4385  576]
 [ 479 4560]]


In [47]:
print(classification_report(y_test, y_pred))

              precision    recall  f1-score   support

    negative       0.90      0.88      0.89      4961
    positive       0.89      0.90      0.90      5039

    accuracy                           0.89     10000
   macro avg       0.89      0.89      0.89     10000
weighted avg       0.89      0.89      0.89     10000



In [48]:
confusion_matrix(y_test, y_pred)

array([[4385,  576],
       [ 479, 4560]], dtype=int64)

In [49]:
clf.predict(['Wow, this is amzing lesson'])

array(['positive'], dtype=object)

In [50]:
clf.predict(['Wow, this sucks'])

array(['negative'], dtype=object)

In [51]:
clf.predict(['Worth of watching it. Please like it'])

array(['positive'], dtype=object)

In [52]:
clf.predict(['Loved it. amazing'])

array(['positive'], dtype=object)